In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans

class RealEstatePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        final_features,
        drop_high_missing=True,
        enforce_property_type=True,
        set_index=True,
        high_missing_threshold=0.5
    ):
        self.final_features = final_features # features that are used in final model
        self.drop_high_missing = drop_high_missing # only use in train set
        self.enforce_property_type = enforce_property_type
        self.set_index = set_index
        self.high_missing_threshold = high_missing_threshold
        # dictionaires to store all intermidiate values
        self.median_values = {}
        self.freq_maps = {}
        self.cap_values = {}
        self.cols_to_drop = []

    def clean_levels(self, val):
        if pd.isna(val):
            return np.nan
        levels = str(val).split(',')
        levels_set = set(levels)
        if levels_set in [{'One', 'Two', 'ThreeOrMore', 'MultiSplit'}, {'MultiSplit'}]:
            return np.nan
        score = 0
        for level in levels:
            if 'Three' in level:
                score = max(score, 3)
            elif 'Two' in level:
                score = max(score, 2)
            elif 'One' in level:
                score = max(score, 1)
        return score if score > 0 else np.nan

    def create_latlon_cluster(self, df, n_clusters=10):
        model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        df["LatLonCluster"] = model.fit_predict(df[["Latitude", "Longitude"]].fillna(0))
    
    def create_latlon_bins(self, df, lat_bin_size=0.1, lon_bin_size=0.1):
        df["LatBin"] = (df["Latitude"] // lat_bin_size) * lat_bin_size
        df["LonBin"] = (df["Longitude"] // lon_bin_size) * lon_bin_size

    def fit(self, X, y=None):
        df = X.copy()

        # Filter by property type if needed
        if self.enforce_property_type:
            df = df[(df["PropertyType"] == "Residential") & (df["PropertySubType"] == "SingleFamilyResidence")]

        # Drop high-missing columns if required
        if self.drop_high_missing:
            missing_rate = df.isna().mean()
            self.cols_to_drop = missing_rate[missing_rate > self.high_missing_threshold].index.tolist()
            df.drop(columns=self.cols_to_drop, inplace=True)

        # Save median for numerical imputation
        for col in df.select_dtypes(include=[np.number]):
            self.median_values[col] = df[col].median()

        # Save frequency encodings
        for col in ['City', 'PostalCode', 'CountyOrParish', 'HighSchoolDistrict']:
            if col in df.columns:
                freq = df[col].value_counts(normalize=True)
                self.freq_maps[col] = freq.to_dict()

        # Save cap values (99th percentile)
        cap_cols = ['LivingArea', 'LotSizeSquareFeet', 'BathroomsTotalInteger', 'BedroomsTotal', 'GarageSpaces', 'AssociationFee']
        for col in cap_cols:
            if col in df.columns:
                self.cap_values[col] = df[col].quantile(0.99)

        return self

    def transform(self, X):
        df = X.copy()

        if self.set_index and "ListingKey" in df.columns:
            df.set_index("ListingKey", inplace=True)

        # Filter by property type
        if self.enforce_property_type:
            df = df[(df["PropertyType"] == "Residential") & (df["PropertySubType"] == "SingleFamilyResidence")]

        df.drop(columns=[c for c in self.cols_to_drop if c in df.columns], errors="ignore", inplace=True)

        # Levels column
        if "Levels" in df.columns:
            df["Levels"] = df["Levels"].apply(self.clean_levels).fillna(1).astype(int)

        # Flooring features
        if "Flooring" in df.columns:
            flooring_filled = df["Flooring"].fillna("").astype(str)
            for mat in ['Carpet', 'Tile', 'Wood', 'Laminate', 'Vinyl', 'Stone', 'Concrete', 'Bamboo']:
                df[f"Has{mat}"] = flooring_filled.str.contains(mat, case=False).astype(int)
            df["FlooringMissing"] = df["Flooring"].isna().astype(int)
            df.drop(columns="Flooring", inplace=True)

        # Latitude/Longitude clusters and bins
        if "Latitude" in df.columns and "Longitude" in df.columns:
            df[["Latitude", "Longitude"]] = df[["Latitude", "Longitude"]].fillna(df[["Latitude", "Longitude"]].median())
            self.create_latlon_cluster(df)
            self.create_latlon_bins(df)
            df.drop(columns=["Latitude", "Longitude"], inplace=True)

        # Frequency encoding
        for col, fmap in self.freq_maps.items():
            df[f"{col}Freq"] = df[col].map(fmap).fillna(0)

        # Fill missing with median
        for col, med in self.median_values.items():
            if col in df.columns:
                df[col] = df[col].fillna(med)

        # Cap outliers
        for col, cap in self.cap_values.items():
            if col in df.columns:
                df[col] = np.where(df[col] > cap, cap, df[col])

        # Additional domain-specific caps
        if "ParkingTotal" in df.columns:
            df["ParkingTotal"] = df["ParkingTotal"].mask(df["ParkingTotal"] < 0, np.nan).fillna(0)
            df["ParkingTotal"] = np.minimum(df["ParkingTotal"], 50)

        if "MainLevelBedrooms" in df.columns:
            df["MainLevelBedrooms"] = np.minimum(df["MainLevelBedrooms"], 10)

        if "YearBuilt" in df.columns:
            df["YearBuilt"] = np.minimum(df["YearBuilt"], 2024)

        if "AssociationFee" in df.columns:
            df["AssociationFee"] = np.minimum(df["AssociationFee"], 2000)

        # Final feature selection
        # df_final = df[[f for f in self.final_features if f in df.columns]].copy() # can be use to improve robust, ignore missing features
        df_final = df[self.final_features].copy()
        if "ClosePrice" in df.columns:
            df_final["ClosePrice"] = df["ClosePrice"]
        df_final.dropna(inplace=True)

        return df_final


In [2]:
# winsorizing extreme value

def apply_price_filter_by_quantile(df, lower_quantile=0.01, upper_quantile=0.99, verbose=True):
    '''cutting extreme value beyond give percentile'''
    q_low = df['ClosePrice'].quantile(lower_quantile)
    q_high = df['ClosePrice'].quantile(upper_quantile)
    mask = df['ClosePrice'].between(q_low, q_high)
    if verbose:
        print(f"Applied quantile filter: [{q_low:.0f}, {q_high:.0f}]")
        print(f"Filtered out {(~mask).sum()} samples ({(~mask).mean():.2%})")
    return df[mask].copy(), (q_low, q_high)


In [3]:
# for training
features = ['LivingArea', 'BathroomsTotalInteger', 'LatBin', 'LatLonCluster', 'LonBin',
            'PostalCodeFreq', 'CountyOrParishFreq', 'CityFreq', 'LotSizeSquareFeet', 'HasWood',
            'YearBuilt', 'HighSchoolDistrictFreq', 'AssociationFee', 'ParkingTotal',
            'BedroomsTotal', 'PoolPrivateYN', 'GarageSpaces', 'MainLevelBedrooms',
            'Levels', 'FireplaceYN', 'ViewYN', 'HasCarpet']

train_raw_df = pd.read_csv("filtered_df_dropped.csv")
preprocessor = RealEstatePreprocessor(final_features=features, enforce_property_type=False)
train_processed = preprocessor.fit_transform(train_raw_df)
train_filtered, price_bounds = apply_price_filter_by_quantile(train_processed)

X_train = train_filtered[features]
y_train = np.log1p(train_filtered['ClosePrice'])

Applied quantile filter: [228452, 5750000]
Filtered out 1015 samples (1.99%)


In [4]:
# model with 6 months
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

final_model = RandomForestRegressor(max_depth=20, n_estimators=200, random_state=42)
final_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=200, random_state=42)

In [5]:
train_filtered['ClosePrice'].describe()

count    5.004200e+04
mean     1.088620e+06
std      7.800168e+05
min      2.285710e+05
25%      6.021558e+05
50%      8.500000e+05
75%      1.300000e+06
max      5.750000e+06
Name: ClosePrice, dtype: float64

In [6]:
train_filtered['ClosePrice'][:10]

0      530000.0
1      700000.0
2     1680000.0
3     1195000.0
6      690000.0
7     1950000.0
8     3350000.0
9     3072000.0
10    3205000.0
11    1125000.0
Name: ClosePrice, dtype: float64

In [7]:
# testing with June data
test_raw_df = pd.read_csv("california/testdata2506/CRMLSSold202506.csv")
test_processed = preprocessor.transform(test_raw_df)
test_filtered = test_processed[
    test_processed['ClosePrice'].between(*price_bounds)
].copy()
X_test = test_filtered[features]

/var/folders/52/h1fcgz7923q3ytk60vdbpqk00000gn/T/ipykernel_30086/3624983933.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_raw_df = pd.read_csv("california/testdata2506/CRMLSSold202506.csv")


In [9]:
test_raw_df.columns

Index(['BuyerAgentAOR', 'ListAgentAOR', 'Flooring', 'ViewYN', 'WaterfrontYN',
       'BasementYN', 'PoolPrivateYN', 'OriginalListPrice', 'ListingKey',
       'ListAgentEmail', 'CloseDate', 'ClosePrice', 'ListAgentFirstName',
       'ListAgentLastName', 'Latitude', 'Longitude', 'UnparsedAddress',
       'PropertyType', 'LivingArea', 'ListPrice', 'DaysOnMarket',
       'ListOfficeName', 'BuyerOfficeName', 'CoListOfficeName',
       'ListAgentFullName', 'CoListAgentFirstName', 'CoListAgentLastName',
       'BuyerAgentMlsId', 'BuyerAgentFirstName', 'BuyerAgentLastName',
       'FireplacesTotal', 'AssociationFeeFrequency', 'AboveGradeFinishedArea',
       'ListingKeyNumeric', 'MLSAreaMajor', 'TaxAnnualAmount',
       'CountyOrParish', 'MlsStatus', 'ElementarySchool', 'AttachedGarageYN',
       'ParkingTotal', 'BuilderName', 'PropertySubType', 'LotSizeAcres',
       'SubdivisionName', 'BuyerOfficeAOR', 'YearBuilt', 'StreetNumberNumeric',
       'ListingId', 'BathroomsTotalInteger', 'City', '

In [10]:
test_filtered.columns

Index(['LivingArea', 'BathroomsTotalInteger', 'LatBin', 'LatLonCluster',
       'LonBin', 'PostalCodeFreq', 'CountyOrParishFreq', 'CityFreq',
       'LotSizeSquareFeet', 'HasWood', 'YearBuilt', 'HighSchoolDistrictFreq',
       'AssociationFee', 'ParkingTotal', 'BedroomsTotal', 'PoolPrivateYN',
       'GarageSpaces', 'MainLevelBedrooms', 'Levels', 'FireplaceYN', 'ViewYN',
       'HasCarpet', 'ClosePrice'],
      dtype='object')

In [11]:
test_filtered.ClosePrice.describe()

count    1.264900e+04
mean     1.025083e+06
std      7.318060e+05
min      2.300000e+05
25%      5.700000e+05
50%      8.050000e+05
75%      1.225000e+06
max      5.750000e+06
Name: ClosePrice, dtype: float64

In [13]:
# predicting log(ClosePrice)
y_pred = final_model.predict(X_test)

# actual price
y_pred_actual = np.expm1(y_pred)
y_actual = test_filtered['ClosePrice']
#rmse = mean_squared_error(test_filtered['ClosePrice'], y_pred_actual)**0.5
#mae = mean_absolute_error(test_filtered['ClosePrice'], y_pred_actual)
mape = mean_absolute_percentage_error(y_actual, y_pred_actual)
r2 = r2_score(y_actual, y_pred_actual)
print(f"Test RMSE: {mape:.2f}\n R2: {r2:.2f}")

Test RMSE: 0.21
 R2: 0.79
